In [16]:
def SavetoFile(saveFile,originSentenceDic):
    File = open(saveFile,"a")
    for englishSentence, values in originSentenceDic.items():
        for threeType, value in values.items():
            for description, va in value.items():
                for content in va:
                    content[1] = re.sub('(\s)*]\s{3}','',content[1])#剔除开头
                    content[1] = re.sub('\s*<span','',content[1])#剔除结尾
                    content[1] = re.sub('</b>','',content[1])#剔除</b>
                    content[1] = re.sub('<b>','',content[1])#剔除<b>
                    content[1] = re.sub('<br/>','',content[1])#剔除<br/>
                    content[1] = re.sub('<strong>','',content[1])#剔除<strong>
                    content[1] = re.sub('</strong>','',content[1])#剔除</strong>
                    content[1] = re.sub('<b>','',content[1])#剔除<b>
                    content[1] = re.sub('</a>','',content[1])#剔除</a>
                    content[1] = re.sub('<a.+?href=\"(.+?)\".*>','',content[1])#剔除</a>
                    content[1] = re.sub('\n','',content[1])#剔除</a>
                    content[1] = re.sub('\t','',content[1])#剔除</a>
                    File.writelines(englishSentence+'####'+threeType+'####'+description+'####'+content[0]+'####'+content[1]+'\n')
    File.close()   

In [17]:
#####2017/1/8 #正则表达式修改版
from lxml import etree
import csv
import re
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
def FiterOneFileToTxt(indexName,saveFile):
    originSentenceDic = {}
    originFile = open(indexName) #将数据读到文件流中
    lists = originFile.read().split('!DOCTYPE html PUBLIC')      #按照split将txt分割成多个HTML列表
    suggestSum = 0
    warnSum = 0
    errorSum = 0 
    for list in lists:
        dom = etree.HTML(list)  #补齐缺失标记
        if dom is not None:
            commentRows = dom.xpath('//div[@class = "view3Item "]')
            #print CommentRows
            for commentrow in commentRows:
                #print etree.tostring(commentrow,encoding="utf-8")
                #print 'iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii'
                suggestDict = {}
                warnDict = {}
                errorDict = {}
                Dict = {}
                originSentenceList = []
                originSentenceList = commentrow.xpath('./div/span/text()')#返回值为List，实际只有一个值
                #此处有点小bug,会存在返回值为0的情况，及里面不存在该选项。
                #r =  originSentenceList[0]
                if len(originSentenceList) > 0:
                    suggs = commentrow.xpath('./div/ul/li[@class= "error_trp snt_hx"]') #获得suggest标记下的li信息
                    for sugg in suggs:
                        key = re.search(r"\[[\u4e00-\u9fa5]*?\]",etree.tostring(sugg,encoding="utf-8")).group()#正则表达式找到[key]值
                        keyContentDim = re.search("content=\"(.+?)\"",etree.tostring(sugg,encoding="utf-8")).group()#正则表达式找到[key]值
                        keyContent = keyContentDim.replace('content="','').replace('"','')
                        description = re.search(r"\](.|\s)*?<span",etree.tostring(sugg,encoding="utf-8")).group()
                        print keyContent,description
                        if key not in suggestDict:
                            suggestDict[key]  = [] # 若该键值不在suggestDict列表，则添加该值
                        suggestDict[key].append([keyContent,description])#把描述信息分别插入suggestDict列表中
                    Dict["suggest"] = suggestDict
                    #warns = commentrow.xpath('./div/ul/li[@class= "warn snt_hx"]')#获得warn标记下的li信息
                    #errors = commentrow.xpath('./div/ul/li[@class= "error snt_hx"]')#获得error标记下的li信息

                    ##处理警告信息
                    warns = commentrow.xpath('./div/ul/li[@class= "warn snt_hx"]')#获得warn标记下的li信息
                    for warn in warns:
                        key = re.search('\[[\u4e00-\u9fa5]*?\]',etree.tostring(warn,encoding="utf-8")).group()#正则表达式找到[key]值
                        keyContentDim = re.search("content=\"(.+?)\"",etree.tostring(warn,encoding="utf-8")).group()#正则表达式找到[key]值
                        keyContent = keyContentDim.replace('content="','').replace('"','')
                        description = re.search(r"\](.|\s)*?<span",etree.tostring(warn,encoding="utf-8")).group()
                        #正则表达式选择出描述信息内容
                        print keyContent,description
                        if key not in warnDict:
                            warnDict[key]  = [] # 若该键值不在suggestDict列表，则添加该值
                        warnDict[key].append([keyContent,description])#把描述信息分别插入suggestDict列表中
                    Dict["warn"] = warnDict

                    ##处理错误信息
                    errors = commentrow.xpath('./div/ul/li[@class= "error snt_hx"]')#获得error标记下的li信息
                    for error in errors:
                        key = re.search('\[[\u4e00-\u9fa5]*?\]',etree.tostring(error,encoding="utf-8")).group()#正则表达式找到[key]值
                        keyContentDim = re.search("content=\"(.+?)\"",etree.tostring(error,encoding="utf-8")).group()#正则表达式找到[key]值
                        keyContent = keyContentDim.replace('content="','').replace('"','')
                        description = re.search(r"\](.|\s)*?<span",etree.tostring(error,encoding="utf-8")).group()
                        print keyContent,description
                        if key not in errorDict:
                            errorDict[key] = []   # 若该键值不在suggestDict列表，则添加该值
                        errorDict[key].append([keyContent,description])#把描述信息分别插入suggestDict列表中
                    Dict["error"] = errorDict     
                    originSentenceDic[originSentenceList[0]] = Dict  
    SavetoFile(saveFile,originSentenceDic)
    originFile.close()

In [18]:
indexName = 'PG_html26.txt'
saveFile  = 'guanlian26.txt'
FiterOneFileToTxt(indexName,saveFile)

In [43]:
saveFile  = 'guanlianall1.txt'
for textIndex in range(26, 27):
    indexName = 'D:\\pigai\\PG_html'+str(textIndex)+'.txt'
    FiterOneFileToTxt(indexName,saveFile)

In [14]:
import re
ok = 0
saveFile  = 'guanlianall1.txt'
errorContent = "errorContent.txt"
keyList = []
keySet = ()
errorSumFile = "errorSumFile.txt"
errorSumF = open(errorContent,"a")
with open(saveFile) as f:
    for line in f.readlines():
        rightSequenceLen  = len(re.split('####',line))
        if rightSequenceLen == 5:
            keyList.append(re.split('####',line)[4])
#keySet = set(keyList)
for key in keyList:
    errorSumF.write(key)
errorSumF.close()
#len(keySet)
    

        
    

In [33]:
import re
saveFile  = 'guanlianall.txt'

with open(saveFile) as f:
    a = len(f.readlines())
a    


        
    

3568109

In [35]:
1784764 * 4

7139056

In [26]:
import re
saveFile  = 'guanlianall.txt'
a = []
keyList = []
keySet = ()
with open(saveFile) as f:
    for line in f.readlines():
        keyList.append(re.split('\t',line)[4])
keySet = set(keyList)
for aa in keySet:
    a.append(aa)
len(a)

        
    

IndexError: list index out of range